In [30]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time, copy, math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
val_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [32]:
from PIL import Image
from torch.utils.data import Dataset

class CatsDogs(Dataset):
    def __init__(self, files, transform, target=-1):
        self.files = files
        self.transform = transform
        self.target = target # training data will have target, val/test will have -1
            
    def __len__(self):
        return len(self.files)
        
    def __getitem__(self,idx):
        img =  Image.open(self.files[idx])
        tensor = self.transform(img)
        return tensor, self.target
        

In [33]:
import os

test_folder = './data/test'
test_files = [os.path.join(test_folder, image) for image in os.listdir(test_folder)]

In [35]:
from torch.utils.data import DataLoader

test_dataset = CatsDogs(test_files,val_transform)
test_dataloader = DataLoader(test_dataset,batch_size=50,shuffle=False)

In [36]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
                                                # 224x224x3
        self.conv1 = nn.Conv2d(3,16,3,padding=1) # 112x112x16
        self.conv2 = nn.Conv2d(16,32,3,padding=1) # 56x56x32
        self.conv3 = nn.Conv2d(32,64,3,padding=1) # 28x28x64       
        self.conv4 = nn.Conv2d(64,128,3,padding=1) # 14x14x128       
        
        self.pooling = nn.MaxPool2d(2,2)
        
        self.dropout = nn.Dropout(0.2) 
        self.fc1 = nn.Linear(14*14*128,500)
        self.fc2 = nn.Linear(500,1)
        
    def forward(self,x):
        out = self.pooling(F.relu(self.conv1(x)))
        out = self.pooling(F.relu(self.conv2(out)))
        out = self.pooling(F.relu(self.conv3(out)))        
        out = self.pooling(F.relu(self.conv4(out)))
        
        out = out.view(-1,14*14*128)
        
        out = self.dropout(out)
        out = F.relu(self.fc1(out))
        
        out = self.dropout(out)
        out = self.fc2(out) # output class score
        
        return out

In [37]:
import torch.optim as optim

net = Net()

net.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [38]:
net.to(device)
net.eval()
probs = []
for idx, (img, target) in enumerate(test_dataloader):
    img, target = img.to(device), target.to(device)

    with torch.no_grad():
        output = net(img)
        prob = F.sigmoid(output)
        prob.to('cpu')
        probs.extend(prob.tolist())

D:\Software\Anaconda3\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [39]:
import pandas as pd

result = pd.DataFrame()
result['id'] = list(range(1,len(test_files)+1))
result['label'] = [x[0] for x in probs]

In [40]:
result.to_csv('self_created_nn_submission_df.csv',index=False)